In [1]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

from model import SchellingModel
from mesa.batchrunner import BatchRunner

from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
import numpy as np
import math

In [2]:
# Define the model inputs
n = 2
problem = {
    'num_vars': n,
    'names': ['mobil', 'var'],
    'bounds': [[0.001, 0.1],
               [0.005, 0.1]]
}

# Generate samples
param_values = saltelli.sample(problem, 200)

Y = np.zeros([param_values.shape[0]])

with tqdm(param_values.size, disable=False) as pbar:
    for i, X in enumerate(param_values):
        model = SchellingModel(25, 25, 0.0015, X[0], 0.50, X[1], 0)
        while model.running:
            model.step()
        Y[i] = model.gent_time
        pbar.update()


1200it [9:04:06, 62.85s/it] 


In [7]:
# remove infeasible solutions
for i in range(0,Y.size,2*n+2):
    if math.isnan(sum(Y[i:i+2*n+2])):
        Y = np.delete(Y,range(i,i+2*n+2))

In [9]:
# Perform analysis
Si = sobol.analyze(problem, Y, print_to_console=True)

# Print the first-order sensitivity indices
print(Si['S1'])

Parameter S1 S1_conf ST ST_conf
mobil 0.423541 0.284979 0.966683 0.182748
var 0.132475 0.118925 0.589026 0.324706

Parameter_1 Parameter_2 S2 S2_conf
mobil var 0.695842 0.903028
[ 0.4235406   0.13247543]


In [8]:
Y

array([ 27.66666667,   8.16666667,  12.83333333,   9.        ,
        12.58333333,   7.58333333,   4.08333333,   4.75      ,
         2.66666667,   2.33333333,   3.91666667,   4.83333333,
        10.25      ,   9.75      ,  14.66666667,   8.        ,
         5.41666667,   6.16666667,   1.5       ,   2.58333333,
         2.58333333,   3.83333333,   2.41666667,   6.08333333,
        12.5       ,  11.66666667,   7.16666667,   7.58333333,
         7.5       ,  11.08333333,   3.25      ,   4.16666667,
         3.66666667,   2.33333333,   2.83333333,   5.66666667,
         4.25      ,   7.75      ,   7.5       ,   4.33333333,
        14.        ,  18.66666667,  12.25      ,   2.16666667,
         9.5       ,   5.16666667,   2.83333333,   4.33333333,
         8.75      ,   4.25      ,   8.5       ,   4.58333333,
         3.91666667,   4.08333333,   1.5       ,  13.75      ,
        12.41666667,   6.25      ,  18.08333333,  24.83333333,
         7.08333333,   4.66666667,   5.        ,   2.58

In [6]:
Y.size

948